In [ ]:
class Sampler:

    def __init__(self):
        self.seed = 1 # A random seed which will be updated after every use. It is there to ensure reproducibility
        self.paths = [] # A list of the paths where original images/signals are stored

    def add_path(self, path):
        self.paths.append(path)

    def set_paths(self, paths):
        self.paths = paths

    def signal_to_vec(self, signal):

    def vec_to_signal(self, vec):

    def sample(self, N):

    def add_noise(self, signal):

    def drop_pixels(self, signal):


